In [2]:
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import Dataset, DataLoader, ConcatDataset
from torchvision import datasets, transforms
from torchvision.datasets import mnist, CIFAR10
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.model_selection import KFold
import time
import torch.nn.functional as TF
import torch.optim as optim
import os
import math
import matplotlib.pyplot as plt
import pickle

torch.set_num_threads(1)


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [3]:
data = np.loadtxt('data/iyer/iyer.txt')[1:]
flag_line = data[:,1]
# print(flag_line)
neg1_list = []
for i in range(len(flag_line)):
    if (data[i,1] == -1):
        neg1_list.append(i)
data = np.delete(data, neg1_list, axis=0)

all_x = data[:,2:]
all_y = data[:,1] - 1

all_idx = np.arange(len(all_x))
SEED_train_test = 1
np.random.seed(SEED_train_test)
np.random.shuffle(all_idx)
tr_len= 400
x_tr = all_x[all_idx[:tr_len]]
y_tr = all_y[all_idx[:tr_len]]

x_te = all_x[all_idx[tr_len:]]
y_te = all_y[all_idx[tr_len:]]

print(x_tr.shape)
print(x_te.shape)

(400, 12)
(84, 12)


In [4]:
SAVE_DIR = 'Models/'
K_FOLD = 5
kf = KFold(n_splits=K_FOLD)
k_idx = 0
for train_index, val_index in kf.split(x_tr):
    train_x = x_tr[train_index]
    train_y = y_tr[train_index]
    val_x = x_tr[val_index]
    val_y = y_tr[val_index]
    # Train model
    rf_clf = RandomForestClassifier(random_state=42)
    rf_clf.fit(train_x, train_y)
    
    save_path = SAVE_DIR + 'IYRE_RF_K_%d.pkl'%k_idx
    pickle.dump(rf_clf, open(save_path, 'wb'))
    
    rf_val_clf = pickle.load(open(save_path, 'rb'))
    
    y_pred = rf_val_clf.predict(val_x)
    score = metrics.accuracy_score(val_y, y_pred)
    print('K_index:%d,Accuracy Score:%.4f%%'%(k_idx,100*score))
    k_idx += 1

K_index:0,Accuracy Score:80.0000%
K_index:1,Accuracy Score:80.0000%
K_index:2,Accuracy Score:75.0000%
K_index:3,Accuracy Score:77.5000%
K_index:4,Accuracy Score:86.2500%


In [5]:
SAVE_DIR = 'Models/'
K_FOLD = 5
kf = KFold(n_splits=K_FOLD)
k_idx = 0
for train_index, val_index in kf.split(x_tr):
    val_x = x_tr[val_index]
    val_y = y_tr[val_index]
    val_y_onehot = np.eye(10)[val_y.astype(int)]
    
    
    save_path = SAVE_DIR + 'IYRE_RF_K_%d.pkl'%k_idx
    pickle.dump(rf_clf, open(save_path, 'wb'))
    
    rf_val_clf = pickle.load(open(save_path, 'rb'))
    
    y_pred = rf_val_clf.predict(val_x)
    y_prob = rf_val_clf.predict_proba(val_x)
    
    score = metrics.accuracy_score(val_y, y_pred)
    F1_score = metrics.f1_score(val_y, y_pred, average='weighted')
    ROC_AUC_score = metrics.roc_auc_score(val_y_onehot, y_prob,average='samples',multi_class='ovo')
    print('K_idx:%d'%(k_idx))
    print('Accuracy Score:', score)
    print('F1 Score:', F1_score)
    print('ROC AUC Score:', ROC_AUC_score)
    print('Average score:%.4f'%((score+F1_score+ROC_AUC_score)/3))
    k_idx += 1

K_idx:0
Accuracy Score: 1.0
F1 Score: 1.0
ROC AUC Score: 1.0
Average score:1.0000
K_idx:1
Accuracy Score: 1.0
F1 Score: 1.0
ROC AUC Score: 1.0
Average score:1.0000
K_idx:2
Accuracy Score: 1.0
F1 Score: 1.0
ROC AUC Score: 1.0
Average score:1.0000
K_idx:3
Accuracy Score: 1.0
F1 Score: 1.0
ROC AUC Score: 1.0
Average score:1.0000
K_idx:4
Accuracy Score: 0.8625
F1 Score: 0.8606734639629374
ROC AUC Score: 0.9833333333333334
Average score:0.9022


In [6]:
SAVE_DIR = 'Models/'
k_idx = 4

y_te_onehot = np.eye(10)[y_te.astype(int)]
save_path = SAVE_DIR + 'IYRE_RF_K_%d.pkl'%k_idx

rf_val_clf = pickle.load(open(save_path, 'rb'))

y_pred = rf_val_clf.predict(x_te)
y_prob = rf_val_clf.predict_proba(x_te)

score = metrics.accuracy_score(y_te, y_pred)
F1_score = metrics.f1_score(y_te, y_pred, average='weighted')
ROC_AUC_score = metrics.roc_auc_score(y_te_onehot, y_prob,average='samples',multi_class='ovo')
print('K_idx:%d'%(k_idx))
print('Accuracy Score:', score)
print('F1 Score:', F1_score)
print('ROC AUC Score:', ROC_AUC_score)

K_idx:4
Accuracy Score: 0.8333333333333334
F1 Score: 0.8249482280637297
ROC AUC Score: 0.9761904761904762
